In [ ]:
#INSTALL AND IMPORT NECESSARY PACKAGES

!pip install beautifulsoup4
!pip install lxml
!pip install requests
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv 
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import folium
import json
from sklearn.cluster import KMeans

# READ TABLE FROM WIKIPEDIA AND CREATE A DATAFRAME
### MANIPULATE DATAFRAME TO GET NECESSARY TABLE
### READ IN GEOGRAPHICAL LOCATION OF NEIGHBORHOODS IN TORONTO 

In [223]:
#Pull table from Wikipedia
#initialize a dataframe
url = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
data = url
df = pd.DataFrame()

#delete unnecessary tables
del data[1:3]
df = data[0]

#delete NULL 
df = df.dropna()

#Read CSV file and create a new dataframe
geodata = pd.read_csv(r'Geo.csv')
geodata

#Merge geolocation dataframe and main dataframe
d = pd.DataFrame()
d = df.merge(geodata, left_on = 'Postal Code', right_on = 'Postal Code')

#print dataframe
d

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [224]:
#Shape of main dataframe
shape = d.shape
print(shape)

(103, 5)


# USE GEOPY LIBRARY TO FIND AND PRINT THE LONGITUDE / LATITUDE OF TORONTO 

In [225]:
#Geopy library to find the longitude and latitude of Toronto
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="TOexplorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


# USE FOLIUM TO CREATE A MAP OF TORONTO

In [226]:
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start = 10)

Toronto_map

# DEFINE FOURSQUARE INFORMATION (CLIENT ID, CLIENT SECRET, AND VERSION
# INFORMATION HIDDEN FOR PRIVACY

In [228]:
CLIENT_ID = 'SGD2KZBNQOVIDUSTL5OM3V4VG5T1OUKUMFD2RTTTLXBPI0UR' # your Foursquare ID
CLIENT_SECRET = 'IJ4AUCDS5L0ZNDZ2H5PA1CK4FUZ505VRKJZOWBFLIFBPFV0Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SGD2KZBNQOVIDUSTL5OM3V4VG5T1OUKUMFD2RTTTLXBPI0UR
CLIENT_SECRET:IJ4AUCDS5L0ZNDZ2H5PA1CK4FUZ505VRKJZOWBFLIFBPFV0Z


# TEST THE FIRST GEOGRAPHICAL LOCATION IN THE DATAFRAME AND PRINT INFORMATION

In [229]:

nlatitude = d.loc[0, 'Latitude']
nlongitude = d.loc[0, 'Longitude']
hoodname = d.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(hoodname, 
                                                               nlatitude, 
                                                               nlongitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


# CREATE FOURSQUARE LINK USING RADIUS OF 500M AND 100 VENUES
## GET RESULTS AND OUTPUT DATA

In [230]:
limit = 100
radius = 500

FourSquare = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nlatitude, 
    nlongitude, 
    radius, 
    limit)


In [231]:
results = requests.get(FourSquare).json()
results

{'meta': {'code': 200, 'requestId': '5ebc5fe0aba297001c8b0218'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

## CREATE FUNCTION THAT EXTRACTS THE CATEGORY OF THE VENUE

In [232]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## IMPORT JSON TO FLATTEN JSON 
## FILTER COLUMNS / FILTER THE CATEGORY FOR EACH ROW / CLEAN COLUMNS
## PRINT THE TOP NEARBY VENUES

In [233]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


# CREATE DATAFRAME THAT CONTAINS ONLY THE BOROUGH WITH THE STRING TORONTO IN IT
## PRINT THE TOP RESULTS OF NEW DATAFRAME

In [234]:
#Copy dataframe 
df = d

#Create a new dataframe containing borough with Toronto
df = df[df['Borough'].str.contains("Toronto")]
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


# EXPLORE TORONTO 
### CREATE A FUNCTION CALLED
### GETNEARBYVENUES WHICH WILL RETRIVE THE VENUES FROM AN API REQUEST

In [235]:
#Explore Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# CREATE A DATAFRAME CONTAINTING THE NEARBY VENUES IN TORONTO
## PRINT THE SHAPE OF THE VENUE

In [236]:

toronto_venue = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [237]:
print(toronto_venue.shape)
toronto_venue.head()

(1605, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


# GROUP TORONTO VENUS BY NEIGHBORHOOD AND COUNT AND PRINT UNIQUE CATEGORIES

In [238]:
toronto_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
Business reply mail Processing Centre,16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,61,61,61,61,61,61
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [148]:
print('There are {} uniques categories.'.format(len(toronto_venue['Venue Category'].unique())))



There are 228 uniques categories.


# ANALYZE EACH NEIGHBORHOOD 
## ONE HOT ENCODING 
## PRINT SHAPE 
## GROUP TORONTO BY NEIGHBORHOOD

In [149]:
#One Hot Encoding
toronto1hot = pd.get_dummies(toronto_venue[['Venue Category']], prefix = "", prefix_sep = "")

#Add neighborhood column back to dataframe
toronto1hot['Neighborhood'] = toronto_venue['Neighborhood']

#Move neighborhood column to the first column
fixed_columns  = [toronto1hot.columns[-1]] + list(toronto1hot.columns[:-1])
toronto1hot = toronto1hot[fixed_columns]

toronto1hot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [239]:
toronto1hot.shape

(1605, 228)

In [240]:
torontoGrouped = toronto1hot.groupby('Neighborhood').mean().reset_index()


# GET NEIGHBORHOOD, TOP 5 VENUES IN NEIGHBORHOOD AND FREQUENCY

In [241]:
num_top_venues = 5

for hood in torontoGrouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torontoGrouped[torontoGrouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1       Nightclub  0.12
2  Breakfast Spot  0.08
3     Coffee Shop  0.08
4       Pet Store  0.04


----Business reply mail Processing Centre----
                venue  freq
0         Yoga Studio  0.06
1       Auto Workshop  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.18
1       Airport Lounge  0.12
2     Airport Terminal  0.12
3  Rental Car Location  0.06
4      Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant

# FUNCTION THAT RETURNS THE MOST COMMON VENUES

In [242]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# CREATE A DATAFRAME THAT CONSISTS OF THE TOP 10 MOST COMMON VENUES IN A NEIGHBORHOOD

In [243]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torontoGrouped['Neighborhood']

for ind in np.arange(torontoGrouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torontoGrouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Café,Restaurant,Pub,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Climbing Gym,Bar,Bakery,Intersection,Restaurant,Italian Restaurant
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop,Bar,Rental Car Location
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Thai Restaurant,Ice Cream Shop,Burger Joint,Salad Place,Bar,Japanese Restaurant


# CREATE CLUSTER AND PRINT THE TOP RESULTS

In [244]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_group_cluster = torontoGrouped.drop(['Neighborhood'], 1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_group_cluster)

kmeans.labels_[0:10]

torontoGrouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Business reply mail Processing Centre,0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Central Bay Street,0.016393,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.0,0.0,0.0


# INSERT COLUMN CLUSTER LABELS FROM KMEANS LABEL
# PRINT THE HEAD OF THE SORTED NEIGHBORHOOD

In [193]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [245]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Café,Restaurant,Pub,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Climbing Gym,Bar,Bakery,Intersection,Restaurant,Italian Restaurant
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Coffee Shop,Bar,Rental Car Location
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Thai Restaurant,Ice Cream Shop,Burger Joint,Salad Place,Bar,Japanese Restaurant


# REMOVE POSTAL CODE FROM DATAFRAME FOR ANALYSIS OF JUST THE NEIGHBORHOOD

In [200]:
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.drop('Postal Code', 1)
toronto_merged.head()


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Breakfast Spot,Restaurant,Theater,Pub,Chocolate Shop,Dessert Shop
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Arts & Crafts Store,Sandwich Place,Distribution Center,Japanese Restaurant,Italian Restaurant,Burrito Place,Beer Bar
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Italian Restaurant,Restaurant,Japanese Restaurant,Fast Food Restaurant
15,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Cosmetics Shop,Italian Restaurant,Gym,Clothing Store,Beer Bar
19,East Toronto,The Beaches,43.676357,-79.293031,0,Park,Health Food Store,Coffee Shop,Pub,Trail,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Diner


# CREATE A MAP USING FOLIUM AND ADD MARKERS TO THE MAP

In [249]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# ANALYZE THE DIFFERENT CLUSTERS ON THE MAP

In [203]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Café,Breakfast Spot,Restaurant,Theater,Pub,Chocolate Shop,Dessert Shop
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Arts & Crafts Store,Sandwich Place,Distribution Center,Japanese Restaurant,Italian Restaurant,Burrito Place,Beer Bar
9,"Garden District, Ryerson",Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Italian Restaurant,Restaurant,Japanese Restaurant,Fast Food Restaurant
15,St. James Town,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Cosmetics Shop,Italian Restaurant,Gym,Clothing Store,Beer Bar
19,The Beaches,Park,Health Food Store,Coffee Shop,Pub,Trail,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Diner
20,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Café,Restaurant,Pub,Breakfast Spot
24,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Thai Restaurant,Ice Cream Shop,Burger Joint,Salad Place,Bar,Japanese Restaurant
25,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports,Coffee Shop
30,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Gym,Clothing Store,Deli / Bodega,Thai Restaurant,Hotel,Pizza Place,Cosmetics Shop
31,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Park,Brewery,Bar,Bank,Café,Supermarket,Middle Eastern Restaurant,Art Gallery


In [205]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,"Moore Park, Summerhill East",Gym,Playground,Women's Store,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [206]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Roselawn,Garden,Women's Store,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [207]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Rosedale,Park,Trail,Playground,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [208]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Lawrence Park,Bus Line,Swim School,Park,College Gym,College Auditorium,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


# OBSERVATION OF NEIGHBORHOODS:

## CLUSTER 1:
        Cluster 1 seems to be the business neighborhoods. 
        Many different business operate in these neighborhoods. 
        Lots of these neighborhoods consists of cafes and coffee shops 
## CLUSTER 2:
        Cluster 2 consists of Moore Park and Summerhill East. 
        These neighborhoods seem to have lots of gyms and playgrounds in the area. 
## CLUSTER 3:
        Though Roselawn neighborhood has a variety of gardens it resembles many of the other clusters

## CLUSTER 4:
        Rosedale seems to be the neighborhood with the most outdoor activity.
        The 3 three most common venues being parks, trails, and playgrounds

## CLUSTER 5:
        Lawrence Park seems to be a commuting neighborhood for schools. 
        The 4th and 5th most common venues in the area are related to colleges.